# 9. Diction

The term 'diction' generally refers to the stylistic choices that are made by an author while writing a text. A study of the diction of an author may concentrate, among other things, on the words that are chosen. In stylometric research, it can be interesting to study the words that are characteristic of a given author, and to examine how the words that are chosen by one author differ from the words chosen by other authors. 

This notebook explains how you can corpare the words in two different texts, and how you can identify the words that are unique or distinctive. 

## A Term Document Matrix

In studies of the diction in a corpus, it is generally very helpful to make a term-document matrix as a first step. A Term Document Matrix represents documents as tables in which the rows represent the words that occur in the full corpus, and in which the columns correspond to the individual texts in this corpus. The values in this matrix describe the frequencies of the words in the various texts. 

To make such a Term Document Matrix, we can follow the steps below. First, we make a list of all the files in the corpus. 

In [ ]:
import os
import re
from collections import Counter
import tdmh
import pandas as pd

dir = 'Corpus'
corpus = []

for file in sorted(os.listdir(dir)):
    if not( re.search( '^\.' , file) ):
        corpus.append(os.path.join(dir,file))


As a second step, we examine the words the are used in all of these texts. The 'types' (i.e. the unique words that are found in these texts) are saved in a list named `vocabulary`. The code below also makles a dictionary named 'all_frequences' which captures the frequencies of these types in the various texts. 

In [ ]:
def extract_title(book):
    title = os.path.basename(book)
    title = re.sub('[.]txt$', '' , title)
    return title


all_frequencies = dict()
all_words = Counter()
all_lengths = dict()
titles = []

for book in corpus:
    print(book)
    row = []
    
    ## Extract the title from the filename
    title = extract_title(book)
    titles.append(title)
    
    ## Read the full text
    file = open(book,encoding='utf-8')
    full_text = file.read()
    
    ## Find all the words in the text
    words = tdmh.tokenise_remove_stopwords(full_text.lower())
    nr_tokens = len(words)
    all_lengths[title] = nr_tokens
    
    ## Count all the words in the text
    freq_book = Counter(words)
    all_frequencies[title] = freq_book

    ## Update the Counter object named 'all_words'
    all_words.update(words)
        
print('Done!')

vocabulary = list(all_words.keys())

Finally, based on the vocabulary and these frequency lists, we can create a term Document Matrix. Each type in the variable `vocabulary` will become a separate row in this matrix. The columns describe the frequencies of these types in the texts in the corpus. 

In [ ]:
dtm = pd.DataFrame( columns= all_words )
        
for book in titles:
    row = dict()
    freq = all_frequencies[book]
    for w in vocabulary:
        row[w] = freq.get(w,0)/all_lengths[book]
    dtm = dtm.append( row  , ignore_index=True )
    
tdm = dtm.transpose()
tdm.columns = titles

tdm.head()

The Term Document Matrix contain relative frequencies. In each cell, the absolute counts of the types are divided by the total number of tokens in each cell. This form of normalisation was applied to mitigate the impact of the differences in length. 

## Unique words

With such a Term Document Matrix in place, it becomes much easier to identify the words that are unique to a given text. To find the words that are unique to *A Room with a View*, for example, we need to find those words from the novel which are *not* used in any of the other texts in the corpus. In other words, the frequencies of these words need to be zero in all of the other texts. The method `find_unique_words()` in the code below implements a method that can be used to identfy these unique words. 

In [ ]:
def find_unique_words(book,nr_words=50):
    unique = tdm.copy()
    columns_other = list(unique.columns)
    columns_other.remove(book)

    unique['sum_other'] = unique[columns_other].sum(axis=1) 
    unique = unique.query( 'sum_other == 0' )
    unique = unique.query( f'{book} > 0' )
    unique = unique.sort_values(book,ascending=False)[book][:nr_words]
    return list(unique.keys())

title = 'ARoomWithaView'
unique_words = find_unique_words(title)
print(f'{title}\n{unique_words}')
    

The code below loops through all the texts in the corpus and identifies, for each text, the most frequent unique words.

In [ ]:
for text in corpus:
    unique_words = find_unique_words( extract_title(text) )
    print(f'{text}\n{unique_words}')

## Distinctive words

Next to establishing the **unique** words, it can also be interesting to identify those words which are very frequently in one text, in comparison to the other texts in the corpus. 

To identify such distinctive words, we firstly calculate the average frequency of each word in the corpus. This average will function as the 'norm' against which we can measure the various frequencies. To do this, we can make use of the `mean()` method in `pandas`. 

In [ ]:
tdm['average'] = tdm.mean(numeric_only=True,axis=1)
tdm['standard_deviation'] = tdm.std(numeric_only=True,axis=1)

Once we know the average values, we can also assess the differences between the frequencies in individual texts on the one hand and the average for the corpus on the other. 

The cell below adds new columns with the suffix '_distinction' for each text in the corpus. The 'distinction' is calculated by subtracting the average frequency from the frequency that was found in that individual text.

In [ ]:
for book in titles:
    print(book)
    column_name = f'{book}_distinction'
    values = []
    
    for word,freq in tdm.iterrows():

        score = (freq[book] - freq['average'])
        if pd.isna(score):
            score = 0
        values.append(score)
    tdm[column_name] = values


In [ ]:
def most_distinctive_words(df,book): 
    column_name = f'{book}_distinction'
    return df[column_name].sort_values( ascending = False )[:30]

for title in titles:
    print(f'\n{title.upper()}\n')
    for word,count in most_distinctive_words(tdm,title).items():
        print( f'{word} ({count}) ' )

## Defining 'subcorpora'

It may also be the case that you have different 'subcorpora' within your overarching corpus. You may have collected texts by two different authors, texts in two different genres, or texts from different periods. It can be interesting to examine the words that are distinctive for such clusters of texts. 

The code below firstly enables you to define such subcorpora. In the lists that are  named `corpus1` and `corpus2`, you need to list all the texts from the two subcorpora whoe words you want to compare. 

The code in this notebook carries out a comparative analysis of the diction in the novels form the Ninetheenth and the Twentieth century. The former texts are added as an item to `corpus1`, and the second texts are appended to `corpus2`.

In [ ]:
dir = 'Corpus'

corpus1 = [ 'PrideandPrejudice.txt','ThroughtheLookingGlass.txt','ATaleofTwoCities.txt' ]
corpus2 = [ 'HeartofDarkness.txt','ARoomWithaView.txt','Ulysses.txt']


# Calculating frequencies

The code in the cell below reads in the full text of the files that are listed in `corpus1`. In this case, we are dealing with one text file only. Next, we calculate the frequencies of all of these words. These frequencies are stored in a dictionary named `freq1`.

Once the first subcorpus has been processed, the code does the same for the texts in `corpus2`. The word frequencies are placed in a dictionary named `freq2`.

After running this code, the variable `full_text1` will contain the *complete* texts of all the texts in `corpus1`. The dictionary named `freq1` will contain the frequencies of all the words in this full text. 

The variables `full_text2` and `freq2` store the same type of information for the texts in `corpus2`.

In [ ]:
from tdmh import *
from os.path import join
from nltk import word_tokenize

from nltk.corpus import stopwords

stopwords = stopwords.words('english')


def tokenise_remove_stopwords(full_text):
    words = word_tokenize(full_text)
    new_list= []
    for w in words:
        w = w.lower().strip()
        orig = ''
        if w.isalnum() and w not in stopwords:
            new_list.append( w )
    return new_list


freq1 = dict()
full_text1 = ''

for text in corpus1:
    print('Reading ' + text + ' ... ')
    with open( join( dir,text) ) as file_handler:
        full_text1 += file_handler.read() + ' '

words = tokenise_remove_stopwords( full_text1  )

for w in words:
    freq1[w] = freq1.get(w,0) +1
    
        
        
freq2 = dict()
full_text2 = ''
    
for text in corpus2:
    print('Reading ' + text + ' ... ')
    with open( join( dir,text) ) as file_handler:
        full_text2 += file_handler.read() + ' '

words = tokenise_remove_stopwords(  full_text2 )

for w in words:
    freq2[w] = freq2.get(w,0) +1
    
print('Done!')
    


##  Dunning's log likelihood

One of statistical methods that can be used to find such distinctive words is *Dunning's log likelihood*. In short, it analyses the distinctiveness of word in one set of texts compared to the texts in a reference corpus, by calculating probabilities based on word frequencies. A good explanation of the fomula can be found on the [wordHoard](https://wordhoard.northwestern.edu/userman/analysis-comparewords.html#loglike) website. 

Using the frequencies that have been calculated above, the Dunning log likelihood scores are calculated for all of the words that occur both in `corpus1` and `corpus2` in the cell below. The actual calculation takes place in a method named `log_likelihood()`. The scores that are calculated are all stored in a dictionary named `ll_scores`

The formula that is implemented in the `log_likelihood` method returns a number which can either be positive or negative. A postive score indicates that there is a high probability that the word will be used in the first corpus. A negative probability indicates that occurence of the word is more common in the second corpus. The tokens that are assigned the highest scores, in other words, are also most distincive of the first corpus. 

The code below lists the 25 words that are assigned a positive log likelihood score in the first corpus. 

In [ ]:
import math

def log_likelihood( word_count1 , word_count2, total1 , total_2 ):

    a = word_count1
    b = word_count2
    c = total1
    d = total2
 
    perc1 = (a/c)*100
    perc2 = (b/d)*100
    polarity = perc1 - perc2
 
    E1 = c*(a+b)/(c+d)
    E2 = d*(a+b)/(c+d)
    
    ln1 = math.log(a/E1)
    ln2 = math.log(b/E2)
    G2 = 2*((a* ln1) + (b* ln2))
    
    #if polarity < 0:
    #    G2 = -G2
    if a * math.log(a / E1) < 0:
        G2 = -G2

    return G2



ll_scores = dict()

total1 = 0
total2 = 0

for word1 in freq1:
    total1 += freq1[word1]
for word2 in freq2:
    total2 += freq2[word2]

for word in freq1:
    if word in freq2:

        ll_score = log_likelihood( freq1[word] , freq2[word] , total1 , total2 )
        ll_scores[word] = ll_score

max = 25
i = 0 
        
for word in sortedByValue(ll_scores , ascending = False ):
    print( word , ll_scores[word] )
    i += 1
    if i == max: 
        break        

Words with negative log likelihood scores are more likely to appear in the reference corpus (i.e. the second corpus) than in the first corpus. 

The code below lists the 25 words with the highest negative scores. 

In [ ]:
max = 25
i = 0 

for word in sortedByValue(ll_scores ) :
    print( word , ll_scores[word] )
    i += 1
    if i == max:
        break   

## Mann Whitney formula

In a [blogpost on identifying literary diction](https://tedunderwood.com/2011/11/09/identifying-the-terms-that-characterize-an-author-or-genre-why-dunnings-may-not-be-the-best-method/), Ted Underwood argues that Dunning's log likelihood function also has a number of disadvantages. It is sensitive to outliers, for example. He explains that the Mann Whitney ranks test can be a good alternative. 

To perform the Mann-Whitney ranks test, we firstly need to find all the words the two corpora to be compared have in common. Next, we need to divide the full texts of the two corpora to be compared into smaller chuncks, all of the same size. These can be fragments of 500 words, for instance. Next, we need to count the number of times each word occurs in these chunks. Using these counts, we can determine whether the word is more frequent in corpus 1 than in corpus 2 (or vice versa). As a final step, we determine the total number of fragments in which the word is most frequent, both in the first and the second corpus. If it is found, using these steps, that a word is much more common in one of the two corpora, this word can be identified as a distinctive word. The Mann-Whitney ranks test really looks at occurrences across the whole corpus, and it is neutralises the effect of exceptionally high counts in one or two of these chunks.      

The Mann Whitney test can be performed in Python using the `mannwhitneyu()` method from the `scipy.stats` module. 

In [ ]:
from scipy.stats import mannwhitneyu

## make a list of all the words in both corpora
words1 = tokenise_remove_stopwords(full_text1)
words2 = tokenise_remove_stopwords(full_text2)

def divide_into_chunks(words, length):

    chunks=[]
    ## chunk contains dictionaries
    # with word frequencies
    
    for i in range(0, len(words), length):
        counts = dict()
        for j in range(length):
            if i+j < len(words):
                word = words[i+j]
                counts[word] = counts.get(word,0)+1
        chunks.append(counts)
    return chunks


length = 500
chunks1 = divide_into_chunks(words1,length)
chunks2 = divide_into_chunks(words2,length)


# vocab is the union of terms in both sets
all_words = dict()
    
for chunk in chunks1:
    for word in chunk:
        all_words[word]= all_words.get(word,0) + 1
for chunk in chunks2:
    for word in chunk:
        all_words[word]= all_words.get(word,0) + 1
    
rho =  dict()
    
for word in all_words:
        
    a=[]
    b=[]
        
    for chunk in chunks1:
        a.append(chunk.get(word,0))
    for chunk in chunks2:
        b.append(chunk.get(word,0))

    stat,pval=mannwhitneyu(a,b, alternative="two-sided")
    mean =len(chunks1)*len(chunks2)*0.5
    if stat <= mean:
        pval = 0 - pval
            
    rho[word]= ( pval )


The words that are most distinctive in corpus 1 have a negative value.

In [ ]:
print( "The following words are most distinctive in corpus 1:" )  

i = 0
max = 25

for word in sortedByValue( rho ):
    if rho[word] > 0:
        print( f'{word}\t{rho[word]:.22f}' ) 
        i += 1
        if i == max:
            break

The words that are most distinctive in corpus 2 have a negative value. 

In [ ]:
print( "The following words are most distinctive in corpus 2:"  )  

i = 0
max = 25

for word in sortedByValue( rho , ascending = False ) :
    if rho[word] < 0:
        print( f'{word}: {rho[word]:.22f}' ) 
        i += 1
        if i == max:
            break

## Bibliography

* Dunning, Ted, 'Accurate Methods for the Statistics of Surprise and Coincidence', in *Computational Linguistics*, 19:1 (1993).
* Rayson, P. and Garside, R., 'Comparing corpora using frequency profiling', in *Proceedings of the workshop on Comparing Corpora, held in conjunction with the 38th annual meeting of the Association for Computational Linguistics (ACL 2000)* (2000)
* H. Mann and D. Whitney, 'On a Test of Whether one of Two Random Variables is Stochastically Larger than the Other', in *Ann. Math. Statist.*, 1:18 (1947). <https://doi.org/10.1214/aoms/1177730491>
* Adam Kilgarriff, *Comparing Corpora*, in *International Journal of Corpus Linguistics*, 6:1 (2001). <https://doi.org/10.1075/ijcl.6.1.05kil>